## Ahmed Nasser Eltabakh
## G8 Nasr City

### Content Based Recommendations

In the previous notebook, you were introduced to a way of making recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve....

In order to help these users out, let's try another technique: **content based** recommendations. Let's start off where we were in the previous notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls "/content/drive/My Drive/recommender_system_files"

all_recs.p	  movies.dat   reviews_clean.csv
movies_clean.csv  ratings.dat  tests.py


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from IPython.display import HTML
#import progressbar
import tests as t
import pickle


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('/content/drive/My Drive/recommender_system_files/movies_clean.csv')
reviews = pd.read_csv('/content/drive/My Drive/recommender_system_files/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']


all_recs = pickle.load(open("/content/drive/My Drive/recommender_system_files/all_recs.p", "rb"))

In [4]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [5]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [6]:
len(all_recs[2])

21

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did receive 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` Let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [7]:
unique_users = reviews['user_id'].unique().tolist()
len(unique_users)

53968

In [8]:
len(all_recs)
#only 23512 user have recommendations

23512

In [9]:
users_with_recs = list(all_recs.keys())

In [10]:
users_with_all_recs = [] # Store user ids who have all their recommendations in this (10 or more)
users_who_need_recs = [] # Store users who still need recommendations here

for user in unique_users:
    if user in users_with_recs:
        if len(all_recs[user]) >= 10:
            users_with_all_recs.append(user)
        else:
            users_who_need_recs.append(user)
    else:
        #no recommendations for user
        users_who_need_recs.append(user)

In [11]:
len(users_who_need_recs)

31781

In [12]:
# A quick test
assert len(users_with_all_recs) == 22187
print("That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 31781 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest to lowest. You will move through the movies in this order looking for other similar movies.

In [13]:
# create a dataframe similar to reviews, but ranked by rating for each user
reviews_ranked = reviews.sort_values(by='rating', ascending=False)
reviews_ranked[0:100]

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
499038,37313,387914,10,1497817674,2017-06-18 20:27:54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
498972,37297,1877832,10,1401121497,2014-05-26 16:24:57,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
498974,37298,368226,10,1509161183,2017-10-28 03:26:23,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
498975,37299,2090488,10,1366687549,2013-04-23 03:25:49,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499297,37328,2713180,10,1414212916,2014-10-25 04:55:16,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
499161,37321,478970,10,1452336694,2016-01-09 10:51:34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
700709,53019,110912,10,1389999307,2014-01-17 22:55:07,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
499165,37321,1431045,10,1464618501,2016-05-30 14:28:21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [14]:
reviews

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712332,53966,6644200,7,1530049728,2018-06-26 21:48:48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
712333,53966,6684714,5,1516287632,2018-01-18 15:00:32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
712334,53967,816711,8,1371972851,2013-06-23 07:34:11,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
712335,53968,1559547,2,1373287369,2013-07-08 12:42:49,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


### Similarities

We have talked in our first Lecture at ITI about similarity metrics and the difference between dot product vs. cosine similarity vs euclidean distance. Here we will apply what we have learnt. 


In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  




`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot product of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [15]:
# Subset so movie_content is only using the dummy variables for each genre and the 3 century based year dummy columns
movies_array = movies.iloc[:, 4:].values

# Take the dot product to obtain a movie x movie matrix of similarities
dot_prod_movies = np.dot(movies_array, movies_array.T)
dot_prod_movies

array([[3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 2, 1],
       [0, 0, 0, ..., 2, 2, 1],
       [0, 0, 0, ..., 1, 1, 2]])

In [16]:
dot_prod_movies.shape

(31245, 31245)

In [17]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,Film-Noir,Mystery,Adventure,Sport,War,Music,Reality-TV,Adult,Crime,Family,Drama,Talk-Show,Biography,Sci-Fi,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [18]:
# create checks for the dot product matrix
assert dot_prod_movies.shape[0] == 31245
assert dot_prod_movies.shape[1] == 31245
assert dot_prod_movies[0, 0] == np.max(dot_prod_movies[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


### For Each User...


Now you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the cell below, complete each of the functions needed for making content based recommendations.

In [19]:
dot_prod_movies[2]

array([3, 3, 3, ..., 0, 0, 0])

In [20]:
#getting maximums
a = np.array([5,10,4,3,1,0])
np.argsort(a, axis=0)[-4:][::-1] #return inices of top 4 numbers

#fill diagonal with zeros
a = np.array([[1,2],[3,4]])
np.fill_diagonal(a, 0) # fill diagonal with 0
a

array([[0, 2],
       [3, 0]])

In [21]:
a = ['dddd', 'dfff']
b = np.array(a)
b

array(['dddd', 'dfff'], dtype='<U4')

In [22]:
movies.iloc[1,1]

'La sortie des usines Lumière (1895)'

In [23]:
movies[movies['movie_id'] == 8]['movie'][0]

'Edison Kinetoscopic Record of a Sneeze (1894)'

In [24]:
reviews['user_id'].unique()

array([    1,     2,     3, ..., 53966, 53967, 53968])

In [25]:
reviews[reviews['user_id'] == 2].sort_values(by='rating', ascending = False)['movie_id'].values

array([2024544, 1798709, 2726560,  422720,  454876, 2883512, 2361509,
       2294629, 1675434, 1535109, 1454468, 1433811, 1398426, 1390411,
        816711, 3079380, 1431045, 2017038, 1322269, 1103275, 1091191,
        790636, 2381249])

In [34]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by ids
    '''
    # find the row of each movie id
    row_index = movies[movies['movie_id'] == movie_id].index[0]
    
    # find the most similar movie indices - to start I said they need to be the same for all content
    np.fill_diagonal(dot_prod_movies, 0) # fill diagonal with 0
    # top 3 similar movies
    similar_movies_indices = np.argsort(dot_prod_movies[row_index], axis=0)[-3:][::-1]
    
    # pull the movie ids based on the indices
    similar_movies = []
    for movie_index in similar_movies_indices:
      movie_id = movies.iloc[movie_index,0]
      similar_movies.append(movie_id)

    return similar_movies
    

def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    # return a list with movie names for the list of input movie IDs

    movie_lst = []
    for id in movie_ids:
      movie_name = movies[movies['movie_id'] == id][['movie']].iloc[0,0]
      movie_lst.append(movie_name)
   
    return movie_lst


def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    # Create dictionary to return with users and ratings
    recs = {}
        
    # For each user

    # Pull only the reviews the user has seen

        # Look at each of the movies (highest ranked first), 
        # pull the movies the user hasn't seen that are most similar
        # These will be the recommendations - continue until 10 recs 
        # or you have depleted the movie list for the user

    users = reviews['user_id'].unique()
    # make recommendation for first 1000 users
    for user in users[:1000]:
      rec_movies = []
      # rated movies by this user ranked
      rated_movies = reviews[reviews['user_id'] == user].sort_values(by='rating', ascending = False)['movie_id'].values
      # get similar movies
      for movie in rated_movies:
        similar_movies = find_similar_movies(movie)
        #remove watched ones
        for s in similar_movies:
          if s in rated_movies:
            # not recommend
            pass
          else:
            #recommend
            if (len(rec_movies) == 10):
              rec_movies_names = get_movie_names(rec_movies)
              recs[user] = rec_movies_names
              break
            else:
              rec_movies.append(s)
      rec_movies_names = get_movie_names(rec_movies)
      recs[user] = rec_movies_names

    return recs

In [35]:
recs = make_recs() # recommendation for only 1000 user for computation effects

### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals for whom you were not able to make recommendations.  

In [36]:
# Explore recommendations
recs

{1: ['The Rainmaker (1997)',
  'Objectif: 500 millions (1966)',
  'The Seven-Ups (1973)',
  'The Split (1968)',
  'Final Chapter: Walking Tall (1977)',
  'Game of Death (1978)'],
 2: ['Castles in the Sky (2014)',
  'Churchill (2017)',
  'Paradesi (2013)',
  'Allegro (2005)',
  '2046 (2004)',
  'Code 46 (2003)',
  'Management (2008)',
  'Strangers (2007)',
  'Cairo Time (2009)',
  'Die Unsichtbaren (2017)'],
 3: ['Watchmen (2009)',
  'Paycheck (2003)',
  'The 6th Day (2000)',
  'Austin Powers in Goldmember (2002)',
  'Casino Jack (2010)',
  'Keeping Mum (2005)',
  'Dolly Ki Doli (2015)',
  '2 Days in Paris (2007)',
  'The Lovers (2017)'],
 4: ['Pacific Rim (2013)', 'Hunting the Phantom (2014)', 'After Earth (2013)'],
 5: ['Farewell to the King (1989)',
  'Race for the Yankee Zephyr (1981)',
  'Azhakan (1991)',
  'The Caine Mutiny (1954)',
  'More American Graffiti (1979)',
  'Jariskatsis mama (1965)'],
 6: ['Snow White and the Huntsman (2012)',
  'Shivaay (2016)',
  'Baahubali 2: The Co

In [37]:
# Some characteristics of my content based recommendations
users_without_all_recs = [] #store user ids without recs
users_with_all_recs = [] # users with all recs here
no_recs = [] # users with no recommendations

for user in recs:
  if len(recs[user]) == 10:
    users_with_all_recs.append(user)
  elif len(recs[user]) == 0:
    no_recs.append(user)
  else:
    users_without_all_recs.append(user)

print("There were {} users without all 10 recommendations we would have liked to have.".format(len(users_without_all_recs)))
print("There were {} users with all 10 recommendations we would like them to have.".format(len(users_with_all_recs)))
print("There were {} users with no recommendations at all!".format(len(no_recs)))

There were 649 users without all 10 recommendations we would have liked to have.
There were 351 users with all 10 recommendations we would like them to have.
There were 0 users with no recommendations at all!
